Чтобы было больше времени на выполнение курсовой работы, задание выполнить на наборе данных для соревнования:  

Тестовая выборка - это выборка для применения модели и загрузки на ЛБ.  

1. Обучить алгоритмы LightGBM и XGBoost, получить OOF прогнозы, оценить корреляцию прогнозов на обучающей выборке. Применить модели на тестовую выборку и оценить корреляцию.  

2. Усреднить прогнозы с помощью арифмитического среднего, геометрического среднего и усреднить ранги, сделать выводы о качестве отдельных моделей и о качестве комбинации.  
3. Обучить CatBoost, получить OOF прогнозы и выполнить задание 1 для трех моделей.  

4. Выполнить задание 2 для трех моделей.  
5. (опция) Объединить OOF-прогнозы для трех моделей и обучить алгоритм Логистической регрессии (и любой другой, на ваше усмотрение). Сделать выводы о достигаемом качестве, сравнить достигаемое качество с качеством отдельных моделей и моделей, полученных в п.2 и п.4.  

6. (опция) Обучить алгоритмRandomForest (желательно подтюнить параметры) и добавить к построенным ранее моделям. Выполнить задание 5.

In [1]:
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [237]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import missingno as msno
import lightgbm as lgb
import catboost as cb
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns
from scipy.stats import probplot, ks_2samp
import warnings
from tqdm import tqdm
from typing import List, Tuple
from scipy.stats import ttest_rel
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score,GroupKFold,TimeSeriesSplit
from classes_and_functions import lgb_param_rnd_test,cb_param_rnd_test,make_cross_validation,make_cross_validation_cb
from classes_and_functions import LGBMClassifier_optimize_params
from classes_and_functions import XGBoost_optimize_params,CatBoostClassifier_optimize_params
from classes_and_functions import test_hold_out_rnd
from classes_and_functions import xgboost_cross_validation
from scipy.stats import gmean, rankdata
from  lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import LabelEncoder
import time
import seaborn as sns
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier


warnings.simplefilter("ignore")
%matplotlib inline
pd.set_option('display.max_columns', None)

In [3]:
PATCH = r'D:\train\kagle/'
patch_train = PATCH + 'train.csv'
patch_test = PATCH + 'test.csv'
patch_bki = PATCH + 'bki.csv'
patch_client_profile = PATCH + 'client_profile.csv'
patch_payments = PATCH + 'payments.csv'
patch_applications_history = PATCH + 'applications_history.csv'
patch_samble_submit = PATCH + 'sample_submit.csv'


In [4]:
# Загрузим данные , уже с отобранными признаками
w_train = pd.read_csv(PATCH + 'w_train_cl_1.csv')
w_test = pd.read_csv(PATCH + 'w_test_cl_1.csv')
train= pd.read_csv(PATCH + 'train.csv')
test= pd.read_csv(PATCH + 'test.csv')

In [5]:
%%time
w_train.replace([np.inf, -np.inf], np.nan, inplace=True)
w_test.replace([np.inf, -np.inf], np.nan, inplace=True)
cat_columns = w_train.select_dtypes(exclude=[np.number]).columns.to_list()
w_train[cat_columns] = w_train[cat_columns].astype(str)
w_test[cat_columns] = w_test[cat_columns].astype(str)
w_train[cat_columns] = w_train[cat_columns].astype('category')
w_test[cat_columns] = w_test[cat_columns].astype('category')

Wall time: 217 ms


### Оптимизируем гмпермараметиры модели LightGBM

In [8]:
params_search = {
    "max_depth": (2, 5),
    "num_leaves":(30,100),
    "min_child_weight": (50, 300),
    "subsample_for_bin": (10000, 500000),
    'min_child_samples':(50,100),
    'subsample':(0.85,1),
    'colsample_bytree':(0.5,1),
    'reg_alpha':(0,500),
    'reg_lambda':(0,500)
    
}

In [9]:
lgb_serch_param = LGBMClassifier_optimize_params(w_train,train['TARGET'])

In [10]:
res_lgb_search = lgb_serch_param.run(params_search,n_iter = 3000,init_points=30)

|   iter    |  target   | colsam... | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample | subsam... |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7118   |  0.718    |  2.078    |  77.48    |  158.8    |  59.43    |  165.2    |  102.3    |  0.9429   |  1.568e+0 |
|  2        |  0.7204   |  0.6334   |  3.863    |  76.46    |  83.64    |  65.95    |  92.22    |  392.7    |  0.9781   |  2.522e+0 |
|  3        |  0.7266   |  0.9233   |  2.239    |  75.26    |  66.32    |  59.97    |  48.27    |  63.58    |  0.9395   |  1.207e+0 |
|  4        |  0.6991   |  0.5535   |  2.661    |  67.49    |  166.9    |  44.12    |  320.2    |  241.5    |  0.9258   |  1.996e+0 |
|  5        |  0.7042   |  0.8968   |  3.74     |  58.11    |  225.2    |  97.52    |  250.0    |  444.8    |  0.9012   |  2.879e+0 |
|  6        |  0.7037   |  0.7138   |  3.31     |  88.83    | 

|  37       |  0.7069   |  0.7903   |  4.442    |  98.97    |  134.9    |  42.44    |  235.1    |  52.05    |  0.8852   |  1.415e+0 |
|  38       |  0.6992   |  0.989    |  4.975    |  72.69    |  181.9    |  59.72    |  333.9    |  420.7    |  0.8678   |  1.989e+0 |
|  39       |  0.7232   |  0.5247   |  3.966    |  80.19    |  210.9    |  34.43    |  71.8     |  299.1    |  0.9762   |  4.349e+0 |
|  40       |  0.7016   |  0.8964   |  3.078    |  79.79    |  161.6    |  83.78    |  294.0    |  209.0    |  0.942    |  2.956e+0 |
|  41       |  0.7265   |  0.9093   |  2.208    |  83.59    |  256.5    |  84.63    |  43.36    |  108.1    |  0.9914   |  4.397e+0 |
|  42       |  0.7001   |  0.6285   |  4.695    |  59.46    |  260.2    |  47.06    |  347.6    |  203.9    |  0.9536   |  1.054e+0 |
|  43       |  0.7132   |  0.8756   |  2.049    |  84.61    |  294.4    |  61.62    |  153.4    |  271.9    |  0.9058   |  4.396e+0 |
|  44       |  0.7167   |  0.7214   |  2.067    |  51.55    | 

|  74       |  0.7257   |  0.7946   |  2.715    |  61.99    |  212.0    |  66.53    |  46.58    |  42.75    |  0.9777   |  7.882e+0 |
|  75       |  0.715    |  0.6439   |  2.134    |  90.12    |  164.3    |  52.28    |  139.0    |  30.0     |  0.8895   |  1.211e+0 |
|  76       |  0.7297   |  0.5887   |  3.641    |  54.77    |  178.2    |  38.94    |  18.49    |  487.2    |  0.9096   |  7.831e+0 |
|  77       |  0.7097   |  0.6843   |  3.732    |  67.27    |  102.3    |  31.56    |  196.4    |  260.6    |  0.8934   |  7.819e+0 |
|  78       |  0.694    |  0.8062   |  2.015    |  70.04    |  231.0    |  36.39    |  452.4    |  203.2    |  0.9514   |  4.353e+0 |
|  79       |  0.7268   |  0.7055   |  2.871    |  77.48    |  108.4    |  58.01    |  40.36    |  96.04    |  0.9738   |  1.208e+0 |
|  80       |  0.7273   |  0.8362   |  4.772    |  71.21    |  248.1    |  93.41    |  59.74    |  125.0    |  0.9259   |  7.906e+0 |
|  81       |  0.7294   |  0.6713   |  4.089    |  94.43    | 

|  111      |  0.7174   |  0.934    |  3.666    |  91.05    |  161.1    |  93.64    |  126.7    |  360.8    |  0.8731   |  9.211e+0 |
|  112      |  0.7327   |  0.6319   |  2.536    |  90.46    |  83.78    |  81.26    |  1.293    |  16.73    |  0.9569   |  4.346e+0 |
|  113      |  0.7291   |  0.9623   |  4.185    |  75.93    |  96.32    |  99.04    |  41.52    |  65.13    |  0.9741   |  4.341e+0 |
|  114      |  0.7282   |  0.92     |  4.686    |  80.03    |  184.1    |  35.84    |  49.79    |  88.05    |  0.8558   |  3.247e+0 |
|  115      |  0.7168   |  0.6336   |  4.865    |  83.09    |  80.35    |  76.97    |  129.8    |  109.9    |  0.89     |  3.934e+0 |
|  116      |  0.7262   |  0.8891   |  4.202    |  68.05    |  279.2    |  77.32    |  66.85    |  310.4    |  0.9541   |  4.345e+0 |
|  117      |  0.7293   |  0.797    |  4.964    |  79.79    |  143.8    |  64.39    |  44.98    |  80.56    |  0.9216   |  1.208e+0 |
|  118      |  0.7313   |  0.5555   |  3.304    |  65.08    | 

|  148      |  0.7304   |  0.5523   |  3.609    |  61.44    |  213.6    |  91.33    |  30.04    |  312.5    |  0.8929   |  3.239e+0 |
|  149      |  0.7286   |  0.8418   |  2.76     |  98.54    |  295.3    |  40.42    |  29.42    |  466.9    |  0.9751   |  8.033e+0 |
|  150      |  0.7001   |  0.697    |  2.417    |  89.13    |  71.11    |  33.24    |  318.4    |  256.1    |  0.9523   |  2.114e+0 |
|  151      |  0.7144   |  0.8941   |  3.391    |  55.13    |  277.7    |  37.37    |  150.8    |  142.8    |  0.9167   |  2.101e+0 |
|  152      |  0.6992   |  0.5375   |  4.422    |  92.94    |  84.89    |  71.44    |  410.0    |  274.1    |  0.8779   |  3.972e+0 |
|  153      |  0.6979   |  0.8366   |  2.929    |  80.5     |  251.8    |  76.36    |  373.7    |  141.9    |  0.9433   |  1.677e+0 |
|  154      |  0.7021   |  0.7264   |  3.544    |  74.5     |  204.3    |  37.13    |  263.5    |  117.2    |  0.9178   |  3.224e+0 |
|  155      |  0.7298   |  0.7075   |  2.901    |  60.38    | 

...

|  2849     |  0.7316   |  0.8554   |  3.478    |  91.13    |  295.8    |  77.41    |  1.644    |  420.9    |  0.8582   |  7.284e+0 |
|  2850     |  0.7295   |  0.942    |  4.492    |  70.33    |  245.6    |  94.13    |  3.188    |  77.88    |  0.9459   |  4.395e+0 |
|  2851     |  0.7292   |  0.8897   |  2.014    |  50.15    |  272.6    |  92.76    |  9.475    |  103.4    |  0.9296   |  3.463e+0 |
|  2852     |  0.6982   |  0.8957   |  2.855    |  92.73    |  167.0    |  94.98    |  395.3    |  134.5    |  0.9564   |  2.738e+0 |
|  2853     |  0.7333   |  0.6283   |  3.036    |  86.4     |  88.77    |  41.6     |  4.317    |  149.4    |  0.9773   |  4.052e+0 |
|  2854     |  0.696    |  0.6171   |  3.29     |  89.21    |  293.4    |  63.76    |  458.2    |  492.9    |  0.8573   |  9.695e+0 |
|  2855     |  0.7322   |  0.8572   |  3.31     |  67.09    |  53.64    |  49.3     |  5.432    |  28.21    |  0.9349   |  4.85e+05 |
|  2856     |  0.7322   |  0.9218   |  3.316    |  98.14    | 

|  2886     |  0.7302   |  0.8391   |  2.229    |  71.04    |  230.4    |  92.43    |  2.603    |  405.2    |  0.8742   |  2.96e+05 |
|  2887     |  0.7289   |  0.6433   |  2.146    |  98.05    |  103.6    |  70.22    |  1.074    |  495.8    |  0.8998   |  2.225e+0 |
|  2888     |  0.7305   |  0.5265   |  2.391    |  95.53    |  150.7    |  76.87    |  16.77    |  287.0    |  0.9935   |  3.2e+05  |
|  2889     |  0.7286   |  0.6232   |  2.322    |  72.11    |  252.0    |  97.12    |  9.185    |  107.6    |  0.9702   |  3.199e+0 |
|  2890     |  0.6936   |  0.6141   |  4.181    |  72.85    |  132.0    |  30.42    |  498.0    |  44.58    |  0.9012   |  9.684e+0 |
|  2891     |  0.7313   |  0.7999   |  4.457    |  50.06    |  97.55    |  99.41    |  11.97    |  40.05    |  0.97     |  1.769e+0 |
|  2892     |  0.697    |  0.6787   |  4.183    |  52.77    |  161.4    |  75.49    |  413.2    |  428.0    |  0.8762   |  2.576e+0 |
|  2893     |  0.7318   |  0.9519   |  3.032    |  71.1     | 

|  2923     |  0.7325   |  0.89     |  3.473    |  99.22    |  105.8    |  90.48    |  0.1831   |  234.6    |  0.9758   |  4.716e+0 |
|  2924     |  0.7297   |  0.5414   |  4.933    |  97.34    |  113.0    |  74.46    |  6.742    |  37.52    |  0.9528   |  3.937e+0 |
|  2925     |  0.7306   |  0.5229   |  2.299    |  80.99    |  287.4    |  64.16    |  20.15    |  233.6    |  0.9212   |  2.742e+0 |
|  2926     |  0.7195   |  0.9905   |  4.686    |  59.77    |  85.7     |  45.08    |  101.6    |  359.2    |  0.9369   |  2.558e+0 |
|  2927     |  0.7312   |  0.5778   |  4.97     |  77.27    |  295.9    |  95.07    |  2.063    |  316.8    |  0.9589   |  2.99e+04 |
|  2928     |  0.713    |  0.8505   |  2.971    |  69.6     |  248.8    |  68.3     |  150.2    |  255.3    |  0.9521   |  3.184e+0 |
|  2929     |  0.7304   |  0.7655   |  4.862    |  77.49    |  276.2    |  99.25    |  19.67    |  492.8    |  0.9015   |  3.009e+0 |
|  2930     |  0.7294   |  0.5675   |  2.975    |  59.83    | 

|  2960     |  0.7228   |  0.8229   |  2.698    |  99.98    |  78.27    |  84.24    |  67.29    |  71.98    |  0.904    |  2.979e+0 |
|  2961     |  0.7302   |  0.8682   |  3.592    |  51.82    |  283.1    |  31.2     |  0.8654   |  2.556    |  0.9417   |  8.66e+04 |
|  2962     |  0.7318   |  0.5801   |  3.055    |  96.1     |  86.93    |  59.63    |  10.76    |  81.23    |  0.9657   |  2.415e+0 |
|  2963     |  0.7213   |  0.5941   |  3.166    |  94.58    |  298.4    |  44.3     |  80.35    |  5.198    |  0.9661   |  2.982e+0 |
|  2964     |  0.7309   |  0.9388   |  3.807    |  70.73    |  56.37    |  92.05    |  2.851    |  464.3    |  0.8582   |  3.201e+0 |
|  2965     |  0.7293   |  0.8144   |  2.117    |  62.57    |  261.7    |  59.51    |  3.137    |  428.0    |  0.9796   |  2.416e+0 |
|  2966     |  0.7289   |  0.9198   |  2.714    |  59.72    |  206.3    |  88.8     |  29.45    |  284.8    |  0.9861   |  2.742e+0 |
|  2967     |  0.731    |  0.8115   |  4.886    |  55.09    | 

|  2997     |  0.7204   |  0.7525   |  3.236    |  55.43    |  114.5    |  86.36    |  92.57    |  10.73    |  0.8587   |  4.063e+0 |
|  2998     |  0.7196   |  0.8012   |  2.344    |  91.82    |  274.9    |  91.49    |  88.79    |  28.14    |  0.8983   |  1.692e+0 |
|  2999     |  0.7294   |  0.5089   |  3.999    |  61.05    |  58.84    |  44.23    |  15.99    |  486.0    |  0.8578   |  8.71e+04 |
|  3000     |  0.7165   |  0.5451   |  2.791    |  58.72    |  68.83    |  50.25    |  104.7    |  428.1    |  0.8695   |  1.693e+0 |
|  3001     |  0.7316   |  0.5381   |  2.279    |  99.48    |  172.7    |  80.62    |  3.291    |  343.1    |  0.89     |  4.288e+0 |
|  3002     |  0.7307   |  0.7872   |  3.925    |  73.13    |  52.24    |  33.72    |  12.25    |  453.9    |  0.9182   |  3.205e+0 |
|  3003     |  0.7306   |  0.7442   |  3.973    |  53.08    |  100.8    |  74.73    |  12.32    |  484.3    |  0.9179   |  4.288e+0 |
|  3004     |  0.7221   |  0.5713   |  3.393    |  67.66    | 

In [11]:
res_lgb_search

(0.7337412847894201,
 {'colsample_bytree': 0.6588716471657045,
  'max_depth': 3,
  'min_child_samples': 92,
  'min_child_weight': 96.66826469244467,
  'num_leaves': 91,
  'reg_alpha': 3.038125709570849,
  'reg_lambda': 289.3991388224806,
  'subsample': 0.9149936984372907,
  'subsample_for_bin': 206018},
 <bayes_opt.bayesian_optimization.BayesianOptimization at 0x205115fbfd0>)

In [139]:
params_lgb = {'colsample_bytree': 0.6588716471657045,
 'max_depth': 3,
 'min_child_samples': 92,
 'min_child_weight': 96.66826469244467,
 'num_leaves': 91,
 'reg_alpha': 3.038125709570849,
 'reg_lambda': 289.3991388224806,
 'subsample': 0.9149936984372907,
 'subsample_for_bin': 206018,
 'n_estimators': 1650,
 'learning_rate': 0.05}

In [35]:
model_lgb=LGBMClassifier(**params_lgb)


In [53]:
%%time
cv_strategy = StratifiedKFold(n_splits=5,random_state = 42,shuffle =True)
params_lgb['n_estimators'] = 1650
params_lgb['learning_rate'] = 0.05
model_lgb = LGBMClassifier(**params_lgb)
res =make_cross_validation(w_train,\
                           train['TARGET'],model_lgb, roc_auc_score, cv_strategy)

Fold: 1, train-observations = 88074, valid-observations = 22019
train-score = 0.7471, valid-score = 0.7243
Fold: 2, train-observations = 88074, valid-observations = 22019
train-score = 0.7385, valid-score = 0.7333
Fold: 3, train-observations = 88074, valid-observations = 22019
train-score = 0.7488, valid-score = 0.7327
Fold: 4, train-observations = 88075, valid-observations = 22018
train-score = 0.7454, valid-score = 0.7234
Fold: 5, train-observations = 88075, valid-observations = 22018
train-score = 0.7491, valid-score = 0.7316
CV-results train: 0.7458 +/- 0.004
CV-results valid: 0.7291 +/- 0.004
OOF-score = 0.7286
Wall time: 16.7 s


In [38]:
%%time
for eta in [0.05]:
    for n_estimators in [1650]:
        params_lgb['n_estimators'] = n_estimators
        params_lgb['learning_rate'] = eta
        model_lgb = LGBMClassifier(**params_lgb)
        print(n_estimators,eta)
        print(test_hold_out_rnd(w_train,train['TARGET'],model_lgb))

1550 0.05
(0.7308663404600415, 0.004606820663802992, 0.7237253322972478, 0.7412006281648432)
1600 0.05
(0.7308600887521236, 0.004619086055029996, 0.7236783525369739, 0.7410726488620983)
1650 0.05
(0.7308789266524507, 0.004626377577473339, 0.7238846810676667, 0.7413465847756676)
1700 0.05
(0.7308540454603202, 0.004574485629000015, 0.7239652837260617, 0.7412069300693969)
Wall time: 3min 35s


In [43]:
%%time
for eta in [0.01]:
    for n_estimators in [7000]:
        params_lgb['n_estimators'] = n_estimators
        params_lgb['learning_rate'] = eta
        model_lgb = LGBMClassifier(**params_lgb)
        print(n_estimators,eta)
        print(test_hold_out_rnd(w_train,train['TARGET'],model_lgb))

6500 0.01
(0.7308512955097262, 0.004467375169151043, 0.7234919269986559, 0.7409065486635039)
7000 0.01
(0.7308969285703368, 0.004449608243982362, 0.7236646568213234, 0.7409926934480731)
7500 0.01
(0.730880544303739, 0.004515146817284849, 0.7236360518825379, 0.7411652362187334)
Wall time: 11min 4s


In [140]:
%%time
cv_strategy = StratifiedKFold(n_splits=5,random_state = 41,shuffle =True)
params_lgb['n_estimators'] = 1650
params_lgb['learning_rate'] = 0.05
model_lgb = LGBMClassifier(**params_lgb)
res =make_cross_validation(w_train,\
                           train['TARGET'],model_lgb, roc_auc_score, cv_strategy)

Fold: 1, train-observations = 88074, valid-observations = 22019
train-score = 0.7436, valid-score = 0.7328
Fold: 2, train-observations = 88074, valid-observations = 22019
train-score = 0.7424, valid-score = 0.7317
Fold: 3, train-observations = 88074, valid-observations = 22019
train-score = 0.7435, valid-score = 0.7283
Fold: 4, train-observations = 88075, valid-observations = 22018
train-score = 0.7528, valid-score = 0.7247
Fold: 5, train-observations = 88075, valid-observations = 22018
train-score = 0.7438, valid-score = 0.7294
CV-results train: 0.7452 +/- 0.004
CV-results valid: 0.7294 +/- 0.003
OOF-score = 0.7289
Wall time: 16.5 s


In [141]:
prod_lgb_oof = res[4]

In [24]:
# на LB
model_lgb.fit(w_train,train['TARGET'],verbose = False)
pred = model_lgb.predict_proba(w_test)[:,1]
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = pred
submission_lgb_base.to_csv(PATCH + 'submission_lgb_clients_opt_select.csv',index=False)


# результат на LB - 0.73227


In [54]:
# на LB
params_lgb['n_estimators'] = 7000
params_lgb['learning_rate'] = 0.01
model_lgb_more = LGBMClassifier(**params_lgb)
model_lgb_more.fit(w_train,train['TARGET'],verbose = False)
pred_lgb_more = model_lgb_more.predict_proba(w_test)[:,1]
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = pred
submission_lgb_base.to_csv(PATCH + 'submission_lgb_clients_opt_select_more.csv',index=False)


# результат на LB - 0.0.73227


### Оптимизируем гмпермараметиры модели XGBoost

In [9]:
params_xgb={
    'n_jobs' : 15,
    'max_depth':6,
    'verbosity':0,
    "objective": "binary:logistic",
    "booster": "gbtree",
    "eval_metric": "auc",
    "learning_rate": 0.1,
    "gamma": 10,
    "subsample": 0.85,
    "colsample_bytree": 0.7,
    "colsample_bylevel": 0.632,
    "min_child_weight": 30,
    "reg_alpha": 5,
    "reg_lambda": 5,
    "random_seed": 42,
    'n_estimators':5000,
    'enable_categorical':True,
    
}

In [10]:
params_search_xgb = {
    "max_depth": (2, 8),
    'gamma':(0,500),
    "min_child_weight": (50, 300),
    "subsample_for_bin": (10000, 500000),
    'min_child_samples':(40,100),
    'subsample':(0.75,1),
    'colsample_bytree':(0.5,1),
    'colsample_bylevel':(0.5,1),
    'reg_alpha':(0,800),
    'reg_lambda':(0,800)
    
}

In [21]:
%%time
w_train_xgb = w_train.copy()
w_test_xgb = w_test.copy()

cat_columns = w_train_xgb.select_dtypes(exclude=[np.number]).columns.to_list()

w_train_xgb[cat_columns] = w_train[cat_columns].astype('object')
w_test_xgb[cat_columns] = w_test[cat_columns].astype('object')

w_train_xgb = w_train_xgb.fillna(-99999)
w_test_xgb = w_test_xgb.fillna(-99999)

w_train_xgb[cat_columns] = w_train[cat_columns].astype(str)
w_test_xgb[cat_columns] = w_test[cat_columns].astype(str)

w_train_xgb[cat_columns] = w_train[cat_columns].astype('category')
w_test_xgb[cat_columns] = w_test[cat_columns].astype('category')

lab_enc= LabelEncoder()
for i in cat_columns:    
    w_train_xgb[i] =  lab_enc.fit_transform(w_train_xgb[i])
    w_test_xgb[i]  =  lab_enc.transform(w_test_xgb[i])

Wall time: 364 ms


In [14]:
%%time
cv_strategy = StratifiedKFold(n_splits=5,random_state = 41,shuffle =True)
res_xgb_cv = xgboost_cross_validation(params_xgb, w_train_xgb, train['TARGET'], cv_strategy, categorical = cat_columns)

Tue Nov  2 10:40:35 2021, Cross-Validation, 110093 rows, 33 cols
Fold 1, Valid score = 0.72887
Fold 2, Valid score = 0.73449
Fold 3, Valid score = 0.7268
Fold 4, Valid score = 0.7218
Fold 5, Valid score = 0.72747
Wall time: 37.5 s


In [22]:
xgb_search_param = XGBoost_optimize_params(w_train_xgb, train['TARGET'],cat_columns)

In [25]:
res_xgb_param = xgb_search_param.run(params_search_xgb,n_iter=1000,init_points = 30)

|   iter    |  target   | colsam... | colsam... |   gamma   | max_depth | min_ch... | min_ch... | reg_alpha | reg_la... | subsample | subsam... |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6766   |  0.718    |  0.513    |  274.8    |  4.612    |  65.22    |  132.6    |  163.7    |  495.4    |  0.8249   |  1.407e+0 |
|  2        |  0.6843   |  0.8106   |  0.7646   |  67.29    |  5.081    |  51.07    |  246.3    |  683.2    |  395.4    |  0.9616   |  4.903e+0 |
|  3        |  0.6873   |  0.7526   |  0.5326   |  214.1    |  2.579    |  47.63    |  199.2    |  180.8    |  85.56    |  0.8051   |  1.814e+0 |
|  4        |  0.6278   |  0.7339   |  0.6009   |  320.2    |  4.898    |  70.31    |  146.7    |  634.9    |  464.0    |  0.7906   |  3.534e+0 |
|  5        |  0.6335   |  0.9823   |  0.75     |  444.8    |  4.05     |  74.03    |  156.9    |  349.4    |  621.2    |  0

|  34       |  0.6921   |  0.9148   |  0.9085   |  48.86    |  2.074    |  56.61    |  61.67    |  159.8    |  745.8    |  0.7899   |  1.01e+05 |
|  35       |  0.6855   |  0.5137   |  0.8354   |  227.1    |  2.856    |  88.05    |  199.3    |  178.6    |  773.3    |  0.8783   |  1.69e+05 |
|  36       |  0.6409   |  0.9483   |  0.6034   |  405.1    |  5.72     |  43.64    |  139.8    |  471.3    |  34.25    |  0.9711   |  1.013e+0 |
|  37       |  0.6475   |  0.5867   |  0.8493   |  337.1    |  4.991    |  96.4     |  76.16    |  9.45     |  796.6    |  0.7544   |  1.015e+0 |
|  38       |  0.6881   |  0.9295   |  0.932    |  8.135    |  7.182    |  81.59    |  298.5    |  525.7    |  629.8    |  0.7772   |  8.663e+0 |
|  39       |  0.6906   |  0.6557   |  0.8173   |  129.6    |  6.031    |  96.61    |  284.9    |  149.7    |  774.5    |  0.9291   |  1.676e+0 |
|  40       |  0.6865   |  0.6319   |  0.9791   |  294.0    |  6.585    |  73.89    |  192.8    |  201.2    |  2.432    |  0

|  68       |  0.7048   |  0.6731   |  0.8357   |  58.01    |  4.317    |  95.04    |  133.3    |  32.82    |  96.75    |  0.8725   |  1.058e+0 |
|  69       |  0.692    |  0.7672   |  0.8654   |  133.7    |  5.031    |  99.83    |  237.5    |  50.42    |  570.5    |  0.8103   |  1.058e+0 |
|  70       |  0.7113   |  0.9399   |  0.5765   |  10.19    |  3.154    |  63.49    |  268.6    |  58.9     |  670.6    |  0.8823   |  1.071e+0 |
|  71       |  0.6879   |  0.8781   |  0.656    |  98.6     |  3.661    |  79.96    |  243.5    |  251.3    |  769.6    |  0.8027   |  1.005e+0 |
|  72       |  0.7073   |  0.968    |  0.9139   |  15.88    |  3.302    |  48.85    |  69.42    |  64.62    |  787.5    |  0.94     |  1.069e+0 |
|  73       |  0.6313   |  0.7178   |  0.6747   |  401.4    |  3.406    |  97.01    |  215.5    |  751.4    |  271.3    |  0.9786   |  3.542e+0 |
|  74       |  0.6891   |  0.9919   |  0.7988   |  293.3    |  3.975    |  69.81    |  265.4    |  20.63    |  290.3    |  0

|  102      |  0.6934   |  0.8899   |  0.9287   |  117.4    |  5.881    |  71.29    |  279.8    |  11.96    |  689.6    |  0.993    |  1.069e+0 |
|  103      |  0.7125   |  0.7264   |  0.7903   |  5.357    |  2.183    |  43.26    |  250.9    |  65.31    |  645.4    |  0.964    |  1.078e+0 |
|  104      |  0.6774   |  0.9301   |  0.8383   |  191.9    |  6.922    |  44.09    |  195.3    |  126.5    |  766.9    |  0.7886   |  1.081e+0 |
|  105      |  0.6933   |  0.536    |  0.8095   |  3.936    |  3.434    |  67.13    |  178.9    |  472.3    |  276.7    |  0.8349   |  4.85e+05 |
|  106      |  0.6841   |  0.566    |  0.55     |  407.6    |  2.711    |  99.77    |  135.4    |  61.26    |  35.21    |  0.8376   |  8.476e+0 |
|  107      |  0.6711   |  0.5072   |  0.5898   |  401.3    |  4.053    |  58.38    |  293.3    |  344.2    |  159.6    |  0.8835   |  5.535e+0 |
|  108      |  0.6677   |  0.5091   |  0.567    |  404.5    |  2.745    |  63.63    |  202.7    |  120.6    |  463.4    |  0

|  136      |  0.6413   |  0.5597   |  0.936    |  355.6    |  5.543    |  95.92    |  217.7    |  459.3    |  54.22    |  0.8389   |  3.61e+05 |
|  137      |  0.6999   |  0.9623   |  0.7563   |  106.8    |  5.745    |  74.34    |  265.2    |  50.66    |  35.92    |  0.9577   |  3.062e+0 |
|  138      |  0.6278   |  0.8047   |  0.5427   |  253.2    |  6.688    |  67.43    |  210.1    |  745.6    |  449.9    |  0.789    |  2.692e+0 |
|  139      |  0.6278   |  0.6454   |  0.8345   |  265.5    |  6.891    |  86.27    |  223.3    |  714.8    |  437.1    |  0.8046   |  8.739e+0 |
|  140      |  0.6848   |  0.6891   |  0.8804   |  369.5    |  7.288    |  45.15    |  287.4    |  37.85    |  170.2    |  0.8419   |  3.147e+0 |
|  141      |  0.6448   |  0.7495   |  0.8427   |  285.5    |  5.636    |  93.87    |  114.5    |  164.0    |  712.1    |  0.799    |  1.699e+0 |
|  142      |  0.6346   |  0.745    |  0.67     |  323.1    |  6.373    |  70.68    |  165.5    |  775.0    |  11.4     |  0

...

|  884      |  0.6335   |  0.6301   |  0.9781   |  330.7    |  5.552    |  60.44    |  211.7    |  531.6    |  159.9    |  0.8128   |  2.433e+0 |
|  885      |  0.7095   |  0.8317   |  0.7461   |  35.15    |  2.583    |  41.31    |  101.1    |  9.973    |  20.97    |  0.7766   |  3.673e+0 |
|  886      |  0.7273   |  0.7324   |  0.8382   |  0.04225  |  6.958    |  46.47    |  276.8    |  8.3      |  682.7    |  0.8625   |  4.785e+0 |
|  887      |  0.6883   |  0.7636   |  0.7753   |  405.7    |  6.547    |  56.09    |  197.1    |  139.0    |  4.776    |  0.9378   |  4.316e+0 |
|  888      |  0.6867   |  0.7281   |  0.9264   |  310.3    |  4.132    |  89.36    |  118.1    |  125.3    |  59.19    |  0.8448   |  3.437e+0 |
|  889      |  0.6334   |  0.8388   |  0.9843   |  193.3    |  2.381    |  70.18    |  53.37    |  776.9    |  549.3    |  0.7857   |  3.646e+0 |
|  890      |  0.6909   |  0.5893   |  0.7993   |  53.68    |  4.075    |  96.71    |  257.4    |  193.5    |  532.8    |  0

|  918      |  0.6911   |  0.7159   |  0.6654   |  111.7    |  2.821    |  41.74    |  139.3    |  211.2    |  533.3    |  0.9526   |  1.357e+0 |
|  919      |  0.7121   |  0.5718   |  0.7241   |  19.22    |  6.768    |  70.78    |  143.4    |  33.14    |  354.6    |  0.9504   |  9.036e+0 |
|  920      |  0.7048   |  0.9519   |  0.7587   |  67.31    |  5.656    |  47.5     |  298.1    |  15.26    |  74.16    |  0.797    |  1.756e+0 |
|  921      |  0.7013   |  0.8258   |  0.6172   |  54.84    |  5.021    |  59.9     |  221.6    |  26.02    |  582.3    |  0.8122   |  4.894e+0 |
|  922      |  0.7178   |  0.8248   |  0.8362   |  11.28    |  4.719    |  52.98    |  160.9    |  25.63    |  125.5    |  0.7543   |  3.439e+0 |
|  923      |  0.7109   |  0.716    |  0.676    |  17.71    |  6.072    |  77.81    |  135.2    |  21.71    |  445.3    |  0.8087   |  3.443e+0 |
|  924      |  0.6425   |  0.5727   |  0.9834   |  484.8    |  3.916    |  74.5     |  200.1    |  123.0    |  755.9    |  0

|  952      |  0.6981   |  0.7944   |  0.7415   |  67.83    |  3.323    |  49.16    |  238.6    |  50.75    |  386.1    |  0.8044   |  4.919e+0 |
|  953      |  0.712    |  0.8535   |  0.8018   |  13.3     |  4.631    |  76.44    |  295.0    |  41.77    |  629.6    |  0.8902   |  3.698e+0 |
|  954      |  0.6375   |  0.5056   |  0.7653   |  253.5    |  5.485    |  95.68    |  213.2    |  645.5    |  730.8    |  0.8066   |  1.551e+0 |
|  955      |  0.703    |  0.6593   |  0.6571   |  8.212    |  6.637    |  53.56    |  296.9    |  215.6    |  451.2    |  0.9196   |  3.368e+0 |
|  956      |  0.7021   |  0.6942   |  0.9135   |  15.52    |  2.086    |  65.4     |  72.7     |  144.5    |  604.9    |  0.9132   |  3.378e+0 |
|  957      |  0.6447   |  0.9063   |  0.9063   |  419.0    |  2.35     |  95.79    |  225.2    |  158.7    |  457.1    |  0.9095   |  3.376e+0 |
|  958      |  0.6795   |  0.5748   |  0.822    |  156.1    |  5.68     |  41.56    |  62.65    |  514.5    |  648.4    |  0

|  986      |  0.6428   |  0.9837   |  0.7657   |  404.8    |  5.646    |  41.04    |  240.3    |  248.0    |  799.8    |  0.8356   |  1.77e+05 |
|  987      |  0.7065   |  0.5249   |  0.7701   |  11.64    |  2.516    |  64.27    |  105.4    |  72.79    |  714.4    |  0.7652   |  4.915e+0 |
|  988      |  0.6614   |  0.9268   |  0.5806   |  326.2    |  4.7      |  99.25    |  71.42    |  267.5    |  28.72    |  0.7965   |  1.768e+0 |
|  989      |  0.7025   |  0.9561   |  0.5864   |  45.27    |  3.331    |  47.8     |  267.4    |  39.79    |  453.9    |  0.9448   |  5.708e+0 |
|  990      |  0.6705   |  0.8898   |  0.9504   |  148.2    |  2.562    |  82.44    |  256.1    |  591.1    |  493.7    |  0.8801   |  2.76e+05 |
|  991      |  0.643    |  0.7425   |  0.6218   |  395.7    |  3.879    |  51.72    |  252.6    |  280.3    |  275.8    |  0.9506   |  4.18e+05 |
|  992      |  0.7004   |  0.6903   |  0.5302   |  96.81    |  2.952    |  87.05    |  186.0    |  15.93    |  177.0    |  0

|  1020     |  0.7047   |  0.6373   |  0.7544   |  81.68    |  3.714    |  56.07    |  285.3    |  8.974    |  16.2     |  0.9694   |  3.436e+0 |
|  1021     |  0.6321   |  0.9304   |  0.7325   |  389.2    |  3.141    |  88.49    |  99.3     |  621.2    |  99.29    |  0.8466   |  3.761e+0 |
|  1022     |  0.6257   |  0.7609   |  0.5531   |  432.5    |  3.685    |  64.47    |  129.7    |  677.0    |  473.8    |  0.8141   |  3.88e+05 |
|  1023     |  0.6425   |  0.5354   |  0.6446   |  239.6    |  4.458    |  83.17    |  169.5    |  716.8    |  62.89    |  0.9777   |  1.783e+0 |
|  1024     |  0.7114   |  0.5751   |  0.518    |  22.87    |  7.885    |  91.42    |  154.9    |  5.947    |  626.1    |  0.8369   |  3.776e+0 |
|  1025     |  0.6654   |  0.5416   |  0.9575   |  269.3    |  6.066    |  99.88    |  244.2    |  160.0    |  722.2    |  0.771    |  3.774e+0 |
|  1026     |  0.7098   |  0.5781   |  0.7203   |  20.08    |  4.697    |  59.38    |  154.9    |  108.8    |  8.311    |  0

In [26]:
res_xgb_param

(0.7293754027320578,
 {'colsample_bylevel': 0.9889316871925483,
  'colsample_bytree': 0.5932978217022177,
  'gamma': 2.441171231031214,
  'max_depth': 5,
  'min_child_samples': 95,
  'min_child_weight': 62.815809354071924,
  'reg_alpha': 25.297624073502334,
  'reg_lambda': 76.27743224563277,
  'subsample': 0.834565450691465,
  'subsample_for_bin': 250935},
 <bayes_opt.bayesian_optimization.BayesianOptimization at 0x25c102c0940>)

In [142]:
params_xgb={
    'n_jobs' : 15,
    'max_depth':6,
    'verbosity':0,
    "objective": "binary:logistic",
    "booster": "gbtree",
    "eval_metric": "auc",
    "random_seed": 42,
    'enable_categorical':True,
    
    'colsample_bylevel': 0.9889316871925483,
    'colsample_bytree': 0.5932978217022177,
    'gamma': 2.441171231031214,
    'max_depth': 5,
    'min_child_samples': 95,
    'min_child_weight': 62.815809354071924,
    'reg_alpha': 25.297624073502334,
    'reg_lambda': 76.27743224563277,
    'subsample': 0.834565450691465,
    'subsample_for_bin': 250935,

    "learning_rate": 0.05,
    'n_estimators':1500,
   
    
}



In [13]:
%%time
cv_strategy = StratifiedKFold(n_splits=5,random_state = 41,shuffle =True)
res_xgb_cv = xgboost_cross_validation(params_xgb, w_train_xgb, train['TARGET'], cv_strategy, categorical = cat_columns)

Thu Nov  4 00:59:04 2021, Cross-Validation, 110093 rows, 33 cols
Fold 1, Valid score = 0.73055
Fold 2, Valid score = 0.73289
Fold 3, Valid score = 0.72577
Fold 4, Valid score = 0.72055
Fold 5, Valid score = 0.72838
oof_score -  0.7274148075606729
Wall time: 56.6 s


In [32]:
%%time
for eta in [0.05]:
    for n_estimators in [700,1000,1500,2000,3000]:
        params_xgb['n_estimators'] = n_estimators
        params_xgb['learning_rate'] = eta
        model_xgb=xgb.XGBClassifier(**params_xgb)
        print(n_estimators,eta)
        print(test_hold_out_rnd(w_train_xgb,train['TARGET'],model_xgb))

700 0.05
(0.7293993693290044, 0.004556642062762831, 0.7199969918717468, 0.7381219930560888)
1000 0.05
(0.7296952133751953, 0.004393644928398302, 0.7203166020709743, 0.738460045221792)
1500 0.05
(0.7299055513586679, 0.004285588436782877, 0.7213543240541123, 0.7381614924935594)
2000 0.05
(0.7298828349301169, 0.004407074412977102, 0.7208456295488685, 0.7381741807031743)
3000 0.05
(0.7299158691095046, 0.0044314298606841825, 0.7209931155817754, 0.7385337127980146)
Wall time: 55min 31s


In [12]:
%%time
for eta in [0.01]:
    for tqdm(n_estimators in [2000,3000,4000,5000,6000]):
        params_xgb['n_estimators'] = n_estimators
        params_xgb['learning_rate'] = eta
        model_xgb=xgb.XGBClassifier(**params_xgb)
        print(n_estimators,eta)
        print(test_hold_out_rnd(w_train_xgb,train['TARGET'],model_xgb))

2000 0.01
(0.7280790664824061, 0.004617891515088338, 0.7197608935649089, 0.7391457712110432)
3000 0.01
(0.7291315230600847, 0.004531928940480929, 0.7209473218335268, 0.739488170002878)
4000 0.01
(0.7296242642328773, 0.004517465321973524, 0.7212337384206257, 0.7398083151942575)
5000 0.01
(0.7298078564648625, 0.004470466500958092, 0.7212866754451729, 0.7393662394031646)
6000 0.01
(0.7299310655618632, 0.0044690069066680525, 0.7217788485532192, 0.7397862163280658)
Wall time: 2h 13min 9s


In [143]:
%%time
cv_strategy = StratifiedKFold(n_splits=5,random_state = 41,shuffle =True)
params_xgb['n_estimators'] = 1500
params_xgb['learning_rate'] = 0.05
res_xgb_cv = xgboost_cross_validation(params_xgb, w_train_xgb, train['TARGET'], cv_strategy, categorical = cat_columns)

Fri Nov  5 02:13:03 2021, Cross-Validation, 110093 rows, 33 cols
Fold 1, Valid score = 0.73055
Fold 2, Valid score = 0.73289
Fold 3, Valid score = 0.72577
Fold 4, Valid score = 0.72055
Fold 5, Valid score = 0.72838
oof_score -  0.7274148075606729
Wall time: 57.3 s


In [171]:
prod_xgb_oof = res_xgb_cv[3]

In [22]:
# на LB
model_xgb.fit(w_train_xgb,train['TARGET'],verbose = False)
pred = model_xgb.predict_proba(w_test_xgb)[:,1]
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = pred
submission_lgb_base.to_csv(PATCH + 'submission_xgb_clients_opt_select.csv',index=False)


# результат на LB - 0.73180

In [55]:
# на LB
params_xgb['n_estimators'] = 7000
params_xgb['learning_rate'] = 0.01
model_xgb_more=xgb.XGBClassifier(**params_xgb)
model_xgb.fit(w_train_xgb,train['TARGET'],verbose = False)
pred_xgb_more = model_xgb.predict_proba(w_test_xgb)[:,1]
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = pred
submission_lgb_base.to_csv(PATCH + 'submission_xgb_clients_opt_select_more.csv',index=False)


# результат на LB - 0.73227

### Оценим кореляцию на обучаюшей и тестовой выборке

In [109]:
scores = pd.DataFrame({
    "lgbm": prod_lgb_oof,
    "xgb": prod_xgb_oof,
})

corr = scores.corr()
corr

,lgbm,xgb
lgbm,1.000000,0.985267
xgb,0.985267,1.000000


In [25]:

prod_test_xgb = model_xgb.predict_proba(w_test_xgb)[:,1]

In [26]:
prod_test_lgbm = model_lgb.predict_proba(w_test)[:,1]

In [27]:
scores_test = pd.DataFrame({
    "lgbm_test": prod_test_xgb,
    "xgb_test": prod_test_lgbm,
})

corr = scores_test.corr()
corr

,lgbm_test,xgb_test
lgbm_test,1.00000,0.97981
xgb_test,0.97981,1.00000


### 2 задание Усреднить прогнозы с помощью арифмитического среднего, геометрического среднего и усреднить ранги, сделать выводы о качестве отдельных моделей и о качестве комбинации.

In [112]:
scores_mean = scores_test.mean(axis=1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_clients_mean.csv',index=False)


# скор на LB 0,73287

In [60]:

scores_mean = scores_test.lgbm_test*0.6 + scores_test.xgb_test*0.4
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_clients_mean_w.csv',index=False)



# скор на LB 0,73287

In [121]:
scores_mean = gmean(scores_test, axis=1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_clients_gmean.csv',index=False)


# скор на LB 0,73280

In [50]:
scores_mean = (scores_test.rank()/len(scores_test)).mean(axis = 1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_clients_rank.csv',index=False)

# скор на LB 0.73292

In [52]:
scores_mean = gmean(scores_test.rank()/len(scores_test),axis = 1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_clients_rank_gmean.csv',index=False)


# скор на LB 0.7328`

In [57]:
scores_test = pd.DataFrame({
    "lgbm_test":  prod_test_lgbm,
    "xgb_test": prod_test_xgb,
    "lgbm_test_more": pred_lgb_more,
    "xgb_test_more": pred_xgb_more,
})

corr = scores_test.corr()
corr

,lgbm_test,xgb_test,lgbm_test_more,xgb_test_more
lgbm_test,1.000000,0.979810,0.998093,0.979810
xgb_test,0.979810,1.000000,0.980805,1.000000
lgbm_test_more,0.998093,0.980805,1.000000,0.980805
xgb_test_more,0.979810,1.000000,0.980805,1.000000


In [58]:
scores_mean = scores_test.mean(axis=1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_clients_mean_4.csv',index=False)


# скор на LB 0.73279

In [61]:
scores_mean = gmean(scores_test, axis=1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_clients_gmean_4.csv',index=False)


# скор на LB 0.73276

In [62]:
scores_mean = (scores_test.rank()/len(scores_test)).mean(axis = 1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_clients_rank_4.csv',index=False)

# скор на LB 0.73274

In [ ]:
scores_mean = scores_test[["lgbm_test",'lgbm_test_more','xgb_test_more']].mean(axis = 1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_clients_mean_3.csv',index=False)

# скор на LB 0.73252

### Задание 3. Обучить CatBoost, получить OOF прогнозы и выполнить задание 1 для трех моделей.

In [66]:
params_search = {
    "max_depth": (2, 5),
    'subsample':(0.85,1),
    'l2_leaf_reg':(0.001,1000),
    'min_child_samples':(10,200),  
    'random_strength':(0.5,1),
    'bagging_temperature':(0.001,500)
  
}

In [67]:
cbb_serch_param = CatBoostClassifier_optimize_params(w_train,train['TARGET'],cat_columns)

In [68]:
res_cb_search = cbb_serch_param.run(params_search,n_iter = 1000,init_points=20)

|   iter    |  target   | baggin... | l2_lea... | max_depth | min_ch... | random... | subsample |
-------------------------------------------------------------------------------------------------
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 217.99801507609976, 'l2_leaf_reg': 25.927205901659505, 'max_depth': 3, 'min_child_samples': 92, 'random_strength': 0.7101839010437445, 'subsample': 0.8995502231505811}
|  1        |  0.7289   |  218.0    |  25.93    |  3.649    |  92.71    |  0.7102   |  0.8996   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 102.325112370287

|  14       |  0.7282   |  398.7    |  269.4    |  3.748    |  14.85    |  0.8311   |  0.9081   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 248.5374022934576, 'l2_leaf_reg': 414.9064224688498, 'max_depth': 3, 'min_child_samples': 114, 'random_strength': 0.9864553449374125, 'subsample': 0.8669164322857107}
|  15       |  0.7261   |  248.5    |  414.9    |  3.053    |  114.7    |  0.9865   |  0.8669   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 156.62995095632837, 'l2_leaf_reg': 41.79866801760024, 'max_depth': 4, 'min_child_samples': 134, 'random_strength': 

|  28       |  0.7266   |  108.6    |  666.8    |  2.991    |  66.18    |  0.5955   |  0.9273   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 469.121491624628, 'l2_leaf_reg': 394.7661916439935, 'max_depth': 3, 'min_child_samples': 52, 'random_strength': 0.7653486794179574, 'subsample': 0.8584680668679797}
|  29       |  0.7282   |  469.1    |  394.8    |  3.459    |  52.11    |  0.7653   |  0.8585   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 232.8625727766831, 'l2_leaf_reg': 377.67704894442136, 'max_depth': 2, 'min_child_samples': 105, 'random_strength': 0.

|  42       |  0.7248   |  105.3    |  480.6    |  2.19     |  101.3    |  0.5822   |  0.9807   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 290.79714395242667, 'l2_leaf_reg': 602.6065172648251, 'max_depth': 2, 'min_child_samples': 172, 'random_strength': 0.8961182906714984, 'subsample': 0.9434506480992488}
|  43       |  0.7257   |  290.8    |  602.6    |  2.882    |  172.7    |  0.8961   |  0.9435   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 41.11162133593575, 'l2_leaf_reg': 641.6480041783584, 'max_depth': 3, 'min_child_samples': 68, 'random_strength': 0

|  56       |  0.7255   |  22.51    |  379.1    |  3.872    |  164.6    |  0.739    |  0.9757   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 451.16859473200526, 'l2_leaf_reg': 702.6663410174973, 'max_depth': 3, 'min_child_samples': 182, 'random_strength': 0.6788963463771372, 'subsample': 0.9090602184084479}
|  57       |  0.7267   |  451.2    |  702.7    |  3.439    |  182.8    |  0.6789   |  0.9091   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 178.54965874933092, 'l2_leaf_reg': 505.0040516452539, 'max_depth': 4, 'min_child_samples': 87, 'random_strength': 

...

{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 408.84659065755005, 'l2_leaf_reg': 385.5637265493654, 'max_depth': 3, 'min_child_samples': 157, 'random_strength': 0.7815673816707716, 'subsample': 0.9070821092294267}
|  966      |  0.7282   |  408.8    |  385.6    |  3.997    |  157.8    |  0.7816   |  0.9071   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 239.77322104500772, 'l2_leaf_reg': 354.9300249699627, 'max_depth': 4, 'min_child_samples': 164, 'random_strength': 0.785167683838434, 'subsample': 0.8994935302597348}
|  967      |  0.7295   |  239.8    |  354.9 

{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 349.0559750782081, 'l2_leaf_reg': 689.891396721358, 'max_depth': 4, 'min_child_samples': 28, 'random_strength': 0.8619997977344542, 'subsample': 0.9941599874154212}
|  980      |  0.7297   |  349.1    |  689.9    |  4.935    |  28.6     |  0.862    |  0.9942   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 429.16659487425227, 'l2_leaf_reg': 467.5062435538313, 'max_depth': 2, 'min_child_samples': 130, 'random_strength': 0.672307888047173, 'subsample': 0.8944989573322478}
|  981      |  0.7256   |  429.2    |  467.5    

{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 25.79707372985143, 'l2_leaf_reg': 421.8314169011019, 'max_depth': 2, 'min_child_samples': 163, 'random_strength': 0.6240263204090787, 'subsample': 0.8687789085897873}
|  994      |  0.725    |  25.8     |  421.8    |  2.815    |  163.4    |  0.624    |  0.8688   |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 180.8969501322107, 'l2_leaf_reg': 104.24985320431034, 'max_depth': 2, 'min_child_samples': 67, 'random_strength': 0.8512091687371937, 'subsample': 0.9357011070922931}
|  995      |  0.7253   |  180.9    |  104.2  

{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 288.1290875833833, 'l2_leaf_reg': 385.6893275060729, 'max_depth': 4, 'min_child_samples': 45, 'random_strength': 0.7204654793741283, 'subsample': 0.9250170955700922}
|  1008     |  0.7294   |  288.1    |  385.7    |  4.29     |  45.94    |  0.7205   |  0.925    |
{'n_estimators': 5000, 'learning_rate': 0.1, 'thread_count': 15, 'verbose': False, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'cat_features': ['NAME_CONTRACT_TYPE', 'EDUCATION_LEVEL', 'GENDER', 'FAMILY_STATUS'], 'bagging_temperature': 348.7227586262564, 'l2_leaf_reg': 993.9858209190973, 'max_depth': 4, 'min_child_samples': 143, 'random_strength': 0.810983642899085, 'subsample': 0.8983533772398216}
|  1009     |  0.7294   |  348.7    |  994.0    

AttributeError: 'CatBoostClassifier_optimize_params' object has no attribute 'lgm_params_search'

In [71]:
cbb_serch_param.best_params,cbb_serch_param.best_target

({'bagging_temperature': 173.13183797538795,
  'l2_leaf_reg': 69.15362818742241,
  'max_depth': 4,
  'min_child_samples': 19,
  'random_strength': 0.7772746659357627,
  'subsample': 0.986185906657642},
 0.7317826366404772)

In [120]:
params_cb = {
        "loss_function": "Logloss",     
        'verbose':False , 
        "eval_metric": "AUC",
        "thread_count": 15, 
        "early_stopping_rounds": 50,
        "random_seed": 27, 
        'cat_features' : cat_columns,
    
    
        "max_depth": 4,
        'subsample' :0.986185906657642,          
        'l2_leaf_reg':69.15362818742241,      
        'min_child_samples':19,    
        'random_strength':0.7772746659357627,
        'bagging_temperature':173.13183797538795,
    
    
         'n_estimators':2500 ,
        'learning_rate':0.05, }


In [121]:
 model_cb=cb.CatBoostClassifier(**params_cb)

In [135]:
%%time
cv_strategy = StratifiedKFold(n_splits=5,random_state = 42,shuffle =True)
params_cb['n_estimators'] = 2500
params_cb['learning_rate'] = 0.05
model_cb=cb.CatBoostClassifier(**params_cb)
res =make_cross_validation_cb(w_train,\
                           train['TARGET'],model_cb, roc_auc_score, cv_strategy)

In [ ]:
%%time
for eta in [0.05]:
    for n_estimators in [2200,2500,3000]:
        params_cb['n_estimators'] = n_estimators
        params_cb['learning_rate'] = eta
        model_cb=cb.CatBoostClassifier(**params_cb)
        print(n_estimators,eta)
        print(test_hold_out_rnd(w_train,train['TARGET'],model_cb,n = 20))
# 1600 0.05        
# (0.730235863655295, 0.0046700176584388445, 0.722357713671054, 0.7393072153149784)
# # Wall time: 57min 8s   
# 2200 0.05
# (0.7304348975465833, 0.004567871988037782, 0.7229545961841468, 0.7383333178599071)

# 2500 0.05
# (0.7304392433061221, 0.004244581751911037, 0.7236189714335128, 0.7378140578047823)

In [ ]:
%%time
for eta in [0.01]:
    for n_estimators in [10000,12000]:
        params_cb['n_estimators'] = n_estimators
        params_cb['learning_rate'] = eta
        model_cb=cb.CatBoostClassifier(**params_cb)
        print(n_estimators,eta)
        print(test_hold_out_rnd(w_train,train['TARGET'],model_cb,n =20))
# 5000 0.01
# (0.7298007601895355, 0.004744136550308015, 0.7209599356803499, 0.7399529635972172)   
# 6000 0.01
# (0.7301579852208254, 0.004607880447184551, 0.7224583810517805, 0.7393165837713016)

In [134]:
%%time
cv_strategy = StratifiedKFold(n_splits=5,random_state = 42,shuffle =True)
params_cb['n_estimators'] = 2500
params_cb['learning_rate'] = 0.05
model_cb=cb.CatBoostClassifier(**params_cb)
res_cb =make_cross_validation_cb(w_train,\
                           train['TARGET'],model_cb, roc_auc_score, cv_strategy)

Fold: 1, train-observations = 88074, valid-observations = 22019
train-score = 0.7435, valid-score = 0.7246
Fold: 2, train-observations = 88074, valid-observations = 22019
train-score = 0.7411, valid-score = 0.7335
Fold: 3, train-observations = 88074, valid-observations = 22019
train-score = 0.7433, valid-score = 0.7295
Fold: 4, train-observations = 88075, valid-observations = 22018
train-score = 0.7423, valid-score = 0.7202
Fold: 5, train-observations = 88075, valid-observations = 22018
train-score = 0.7418, valid-score = 0.7306
CV-results train: 0.7424 +/- 0.001
CV-results valid: 0.7277 +/- 0.005
OOF-score = 0.7273
Wall time: 23min 35s


In [136]:
prod_cb_oof = res[4]

In [148]:
# на LB
model_cb=cb.CatBoostClassifier(**params_cb)
model_cb.fit(w_train,train['TARGET'],verbose = False)
pred_test_cb = model_cb.predict_proba(w_test)[:,1]
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = pred_test_cb
submission_lgb_base.to_csv(PATCH + 'submission_cb_clients_opt_select.csv',index=False)


# результат на LB - 0.73338

In [172]:
scores = pd.DataFrame({
    "lgbm": prod_lgb_oof,
    "xgb": prod_xgb_oof,
    'cb':prod_cb_oof
})

corr = scores.corr()
corr

,lgbm,xgb,cb
lgbm,1.000000,0.978209,0.971373
xgb,0.978209,1.000000,0.972918
cb,0.971373,0.972918,1.000000


In [146]:
scores_test = pd.DataFrame({
    "lgbm_test":  prod_test_lgbm,
    "xgb_test": prod_test_xgb,
    "cb_test": pred_test_cb,
})

corr = scores_test.corr()
corr

,lgbm_test,xgb_test,cb_test
lgbm_test,1.000000,0.979810,0.963326
xgb_test,0.979810,1.000000,0.965219
cb_test,0.963326,0.965219,1.000000


In [151]:
scores_mean = scores_test.mean(axis=1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_cb_clients_mean.csv',index=False)


# скор на LB - 0.73418

In [152]:
scores_mean = gmean(scores_test, axis=1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_cb_clients_gmean.csv',index=False)


# скор на LB 0.73410

In [153]:
scores_mean = (scores_test.rank()/len(scores_test)).mean(axis = 1)
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = scores_mean
submission_lgb_base.to_csv(PATCH + 'submission_xgb_v_lgb_cb__clients_rank.csv',index=False)

# скор на LB  0.73415

Последние задания

In [157]:
%%time
cv_strategy = StratifiedKFold(n_splits=5,random_state = 42,shuffle =True)
params_cb['n_estimators'] = 2500
params_cb['learning_rate'] = 0.05
model_cb=cb.CatBoostClassifier(**params_cb)
res_cb =make_cross_validation_cb(w_train,\
                           train['TARGET'],model_cb, roc_auc_score, cv_strategy)

Fold: 1, train-observations = 88074, valid-observations = 22019
train-score = 0.7435, valid-score = 0.7246
Fold: 2, train-observations = 88074, valid-observations = 22019
train-score = 0.7411, valid-score = 0.7335
Fold: 3, train-observations = 88074, valid-observations = 22019
train-score = 0.7433, valid-score = 0.7295
Fold: 4, train-observations = 88075, valid-observations = 22018
train-score = 0.7423, valid-score = 0.7202
Fold: 5, train-observations = 88075, valid-observations = 22018
train-score = 0.7418, valid-score = 0.7306
CV-results train: 0.7424 +/- 0.001
CV-results valid: 0.7277 +/- 0.005
OOF-score = 0.7273
Wall time: 2min 4s


In [158]:
%%time
cv_strategy = StratifiedKFold(n_splits=8,random_state = 42,shuffle =True)
params_cb['n_estimators'] = 2500
params_cb['learning_rate'] = 0.05
model_cb=cb.CatBoostClassifier(**params_cb)
res_cb =make_cross_validation_cb(w_train,\
                           train['TARGET'],model_cb, roc_auc_score, cv_strategy)

Fold: 1, train-observations = 96331, valid-observations = 13762
train-score = 0.7375, valid-score = 0.7242
Fold: 2, train-observations = 96331, valid-observations = 13762
train-score = 0.7415, valid-score = 0.7288
Fold: 3, train-observations = 96331, valid-observations = 13762
train-score = 0.7373, valid-score = 0.7406
Fold: 4, train-observations = 96331, valid-observations = 13762
train-score = 0.7388, valid-score = 0.7298
Fold: 5, train-observations = 96331, valid-observations = 13762
train-score = 0.7455, valid-score = 0.7256
Fold: 6, train-observations = 96332, valid-observations = 13761
train-score = 0.742, valid-score = 0.7261
Fold: 7, train-observations = 96332, valid-observations = 13761
train-score = 0.7412, valid-score = 0.724
Fold: 8, train-observations = 96332, valid-observations = 13761
train-score = 0.7384, valid-score = 0.7251
CV-results train: 0.7403 +/- 0.003
CV-results valid: 0.728 +/- 0.005
OOF-score = 0.7277
Wall time: 2min 39s


In [161]:
%%time
cv_strategy = StratifiedKFold(n_splits=5,random_state = 41,shuffle =True)
params_xgb['n_estimators'] = 1500
params_xgb['learning_rate'] = 0.05
res_xgb_cv = xgboost_cross_validation(params_xgb, w_train_xgb, train['TARGET'], cv_strategy)

Fri Nov  5 02:40:49 2021, Cross-Validation, 110093 rows, 33 cols
Fold 1, Valid score = 0.7316
Fold 2, Valid score = 0.73432
Fold 3, Valid score = 0.72653
Fold 4, Valid score = 0.72021
Fold 5, Valid score = 0.72867
oof_score -  0.7280262482425377
Wall time: 1min 19s


In [162]:
%%time
cv_strategy = StratifiedKFold(n_splits=8,random_state = 41,shuffle =True)
params_xgb['n_estimators'] = 1500
params_xgb['learning_rate'] = 0.05
res_xgb_cv = xgboost_cross_validation(params_xgb, w_train_xgb, train['TARGET'], cv_strategy)

Fri Nov  5 02:42:09 2021, Cross-Validation, 110093 rows, 33 cols
Fold 1, Valid score = 0.73697
Fold 2, Valid score = 0.72617
Fold 3, Valid score = 0.73369
Fold 4, Valid score = 0.72485
Fold 5, Valid score = 0.73769
Fold 6, Valid score = 0.70434
Fold 7, Valid score = 0.7375
Fold 8, Valid score = 0.7322
oof_score -  0.7288701075787731
Wall time: 2min 11s


In [166]:
%%time
cv_strategy = StratifiedKFold(n_splits=5,random_state = 41,shuffle =True)
params_lgb['n_estimators'] = 1650
params_lgb['learning_rate'] = 0.05
model_lgb = LGBMClassifier(**params_lgb)
res_lgb =make_cross_validation(w_train,\
                           train['TARGET'],model_lgb, roc_auc_score, cv_strategy)

Fold: 1, train-observations = 88074, valid-observations = 22019
train-score = 0.7436, valid-score = 0.7328
Fold: 2, train-observations = 88074, valid-observations = 22019
train-score = 0.7424, valid-score = 0.7317
Fold: 3, train-observations = 88074, valid-observations = 22019
train-score = 0.7435, valid-score = 0.7283
Fold: 4, train-observations = 88075, valid-observations = 22018
train-score = 0.7528, valid-score = 0.7247
Fold: 5, train-observations = 88075, valid-observations = 22018
train-score = 0.7438, valid-score = 0.7294
CV-results train: 0.7452 +/- 0.004
CV-results valid: 0.7294 +/- 0.003
OOF-score = 0.7289
Wall time: 16.5 s


In [165]:
%%time
cv_strategy = StratifiedKFold(n_splits=8,random_state = 41,shuffle =True)
params_lgb['n_estimators'] = 1650
params_lgb['learning_rate'] = 0.05
model_lgb = LGBMClassifier(**params_lgb)
res_lgb =make_cross_validation(w_train,\
                           train['TARGET'],model_lgb, roc_auc_score, cv_strategy)

Fold: 1, train-observations = 96331, valid-observations = 13762
train-score = 0.7387, valid-score = 0.7387
Fold: 2, train-observations = 96331, valid-observations = 13762
train-score = 0.7478, valid-score = 0.7261
Fold: 3, train-observations = 96331, valid-observations = 13762
train-score = 0.7339, valid-score = 0.7294
Fold: 4, train-observations = 96331, valid-observations = 13762
train-score = 0.7408, valid-score = 0.7258
Fold: 5, train-observations = 96331, valid-observations = 13762
train-score = 0.7439, valid-score = 0.7369
Fold: 6, train-observations = 96332, valid-observations = 13761
train-score = 0.744, valid-score = 0.7059
Fold: 7, train-observations = 96332, valid-observations = 13761
train-score = 0.7401, valid-score = 0.7366
Fold: 8, train-observations = 96332, valid-observations = 13761
train-score = 0.742, valid-score = 0.7322
CV-results train: 0.7414 +/- 0.004
CV-results valid: 0.7289 +/- 0.01
OOF-score = 0.7281
Wall time: 23.2 s


In [173]:
super_train = pd.DataFrame({
    "lgbm": res_lgb[4],
    "xgb": res_xgb_cv[3],
    'cb': res_cb[4]
})

In [212]:
params_lr={'random_state':42,
          'solver':'liblinear'}
scoring = ['roc_auc']

In [224]:
%%time
result_params_lr = pd.DataFrame()
for penalty in ['l1','l2']:
    for C in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,5,10,15,20,25,35,40,45,50,55,60,65]:
        params_lr['penalty'] = penalty
        params_lr['C'] = C
        model_lr = LogisticRegression(**params_lr)
        scores = cross_validate(model_lr, super_train, train.TARGET, cv=5, scoring=scoring)
        result_params_lr = result_params_lr.append({'penalty':params_lr['penalty'], 'C':params_lr['C'],\
                                                    "mean_scor":scores['test_roc_auc'].mean(),\
                                                   "std_scor":scores['test_roc_auc'].std()},\
                                                   ignore_index=True)
        
        
                
                

Wall time: 40.1 s


In [225]:
result_params_lr.sort_values('mean_scor',ascending=False,inplace=True)


In [226]:
result_params_lr.head(10)

,penalty,C,mean_scor,std_scor
32,l2,1.0,0.729717,0.004521
33,l2,2.0,0.729714,0.004519
31,l2,0.9,0.729704,0.004528
30,l2,0.8,0.729694,0.004532
28,l2,0.6,0.729683,0.004566
27,l2,0.5,0.729682,0.004567
29,l2,0.7,0.729680,0.004555
26,l2,0.4,0.729680,0.004575
34,l2,5.0,0.729672,0.004525
25,l2,0.3,0.729666,0.004594


In [230]:
model_lr = LogisticRegression(random_state = 42)
scores = cross_validate(model_lr, super_train, train.TARGET, cv=5, scoring=scoring)
scores['test_roc_auc'].mean()

0.7297184531383132

In [231]:
super_test = pd.DataFrame({
    "lgbm":  prod_test_lgbm,
    "xgb": prod_test_xgb,
    "cb": pred_test_cb,
})

array([[0.94339919, 0.05660081],
       [0.6159772 , 0.3840228 ],
       [0.90612331, 0.09387669],
       ...,
       [0.92906408, 0.07093592],
       [0.96047903, 0.03952097],
       [0.94652001, 0.05347999]])

In [236]:
model_lr.fit(super_train,train.TARGET)
super_score = model_lr.predict_proba(super_test)[:,1]
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = super_score
submission_lgb_base.to_csv(PATCH + 'submission_super_lr_clients.csv',index=False)

# скор на LB  0.0.73382

RandomForestClassifier

In [239]:
model_rf = RandomForestClassifier(random_state=42)
scores = cross_validate(model_rf, super_train, train.TARGET, cv=5, scoring=scoring)
scores['test_roc_auc'].mean()

0.6794224140274235

In [ ]:
random_state=42
n_jobs = 15
n_estimators
max_depth
min_samples_split
min_samples_leaf

In [247]:
%%time
result_params_lr = pd.DataFrame()
for n_estimators in tqdm([300,400]):
    for max_depth in [2,4,6]:
        for min_samples_split in [4,5,6,10]:
            for min_samples_leaf in [2,3,4]:
                params_rf={
                    'random_state':42,
                    "n_jobs" : 15,
                    'n_estimators':n_estimators,
                    'max_depth' :max_depth,
                    "min_samples_split":min_samples_split,
                    'min_samples_leaf' :min_samples_leaf
                    
                }
                
                model_rf = RandomForestClassifier(**params_rf)
                scores = cross_validate(model_rf, super_train, train.TARGET, cv=5, scoring=scoring)
                result_params_lr = result_params_lr.append({'n_estimators':params_rf['n_estimators'], \
                                                    'max_depth':params_rf['max_depth'],\
                                                    'min_samples_split':params_rf['min_samples_split'] ,\
                                                     'min_samples_leaf':params_rf['min_samples_leaf'],        
                                                             
                                                    "mean_scor":scores['test_roc_auc'].mean(),\
                                                   "std_scor":scores['test_roc_auc'].std()},\
                                                   ignore_index=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [08:35<00:00, 257.63s/it]

Wall time: 8min 35s


In [248]:
result_params_lr.sort_values('mean_scor',ascending=False,inplace=True)
result_params_lr.head(10)

,n_estimators,max_depth,min_samples_split,min_samples_leaf,mean_scor,std_scor
29,300.0,6.0,5.0,4.0,0.729009,0.004910
32,300.0,6.0,6.0,4.0,0.729009,0.004910
26,300.0,6.0,4.0,4.0,0.729009,0.004910
15,300.0,4.0,5.0,2.0,0.728988,0.004754
12,300.0,4.0,4.0,2.0,0.728988,0.004754
18,300.0,4.0,6.0,2.0,0.728987,0.004754
21,300.0,4.0,10.0,2.0,0.728987,0.004753
48,400.0,4.0,4.0,2.0,0.728956,0.004688
51,400.0,4.0,5.0,2.0,0.728956,0.004688
34,300.0,6.0,10.0,3.0,0.728956,0.004950


In [246]:
params_rf={
                    'random_state':42,
                    "n_jobs" : 15,
                    'n_estimators':500,
                    'max_depth' :4,
                    "min_samples_split":5,
                    'min_samples_leaf' :2
                    
                }
                

model_rf = RandomForestClassifier(**params_rf)
model_rf.fit(super_train,train.TARGET)
super_score = model_rf.predict_proba(super_test)[:,1]
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = super_score
submission_lgb_base.to_csv(PATCH + 'submission_super_rf_clients.csv',index=False)

# скор на LB  0.73344

In [255]:
import time
import tensorflow as tf
def time_matmul(x):
  start = time.time()
  for loop in range(100):
    tf.matmul(x, x)

  result = time.time()-start

  print("100 циклов: {:0.2f}ms".format(1000*result))

for i in range(2):
    # исполнение CPU
    print(" CPU работает:")
    with tf.device("CPU:0"):
      x = tf.random.uniform([1000, 1000])
      assert x.device.endswith("CPU:0")
      time_matmul(x)

    # исполнение на GPU #0 , если доступен
    if tf.config.experimental.list_physical_devices("GPU"):
      print(" GPU работает:")
      with tf.device("GPU:0"): # или GPU:1 , для 2х GPU, GPU:2 для 3х  и т.д.
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

 CPU работает:
100 циклов: 369.77ms
 GPU работает:
100 циклов: 1852.58ms
 CPU работает:
100 циклов: 342.31ms
 GPU работает:
100 циклов: 6.01ms


In [392]:
input_dim = []
for col_name in super_train:
    input_dim = input_dim + super_train[col_name].to_list()
for col_name in super_test:
    input_dim = input_dim + super_test[col_name].to_list()
input_dim = set(input_dim)
len(input_dim)

669399

In [419]:
inp = tf.keras.layers.Input(shape= (3))
emb = tf.keras.layers.Embedding(input_dim =669400, output_dim = 64,input_length=3)(inp)
x3 = tf.keras.layers.Conv1D(filters=512,kernel_size =3, strides=1,padding='same',\
                            activation = 'relu',input_shape =(3,64))(emb)

x3= tf.keras.layers.GlobalMaxPooling1D()(x3)
x4 = tf.concat([x3,inp],axis=1)
x = tf.keras.layers.Dense(512,activation = 'relu')(x4)
x= tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1024,activation = 'relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512,activation = 'relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
out = x = tf.keras.layers.Dense(2,activation ='softmax' )(x)
model = tf.keras.Model(inp,out)
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 3)]          0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 3, 64)        42841600    input_31[0][0]                   
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 3, 512)       98816       embedding_12[0][0]               
__________________________________________________________________________________________________
global_max_pooling1d_2 (GlobalM (None, 512)          0           conv1d_22[0][0]                  
___________________________________________________________________________________________

In [420]:
metrica = tf.keras.metrics.AUC()

In [421]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrica])

In [422]:
x,x_v, y,y_v = train_test_split(super_train, train.TARGET,\
                                                     test_size= 0.2,random_state = 25)
y = pd.get_dummies(y).to_numpy()
y_v = pd.get_dummies(y_v).to_numpy()

In [424]:
model.fit(x, y,  epochs=1,
          validation_data=(x_v, y_v))


2753/2753 [==============================] - 106s 39ms/step - loss: 0.2608 - auc_13: 0.9470 - val_loss: 0.2510 - val_auc_13: 0.9467


In [425]:
model.evaluate(x_v, y_v)

689/689 [==============================] - 2s 3ms/step - loss: 0.2510 - auc_13: 0.9467


[0.2510084807872772, 0.9467054605484009]

In [426]:
pred_test_n= model.predict(super_test)[:,1]
pred_test_n

array([1.31162221e-03, 1.16155639e-01, 1.17803225e-02, ...,
       4.33363300e-03, 5.70700249e-05, 9.45253822e-04], dtype=float32)

In [427]:
submission_lgb_base = pd.DataFrame()
submission_lgb_base['APPLICATION_NUMBER'] = test['APPLICATION_NUMBER']
submission_lgb_base['TARGET'] = pred_test_n
submission_lgb_base.to_csv(PATCH + 'submission_super_net_clients.csv',index=False)

# скор на LB  0.73405